In [ ]:
!apt-get install unzip

In [ ]:
!pip install accelerate

In [ ]:
!pip install git+https://github.com/Pranavmath/diffusers.git

In [ ]:
!pip install datasets

In [ ]:
!unzip nodule_dataset.zip

In [ ]:
!pip install matplotlib

In [6]:
from PIL import Image
from PIL import ImageOps, ImageDraw
from PIL import ImageFilter
import os
import torch
import json
import numpy as np
import random

In [7]:
!mkdir -p better_dataset/generated_masks
!mkdir -p better_dataset/subtracts

In [8]:
!mkdir -p test/generated_masks
!mkdir -p test/subtracts

In [9]:
SIZE = 256

In [10]:
with open('./nodule_dataset/nodulemetadata.json') as f:
    nodule_dict = json.load(f)

mask_path = "./nodule_dataset/generated_masks"
subtract_path = "./nodule_dataset/subtracts"

for nodule_name, (bbox, _, _) in nodule_dict.items():
  nodule_name = f"{nodule_name}.jpg"

  mask = Image.open(os.path.join(mask_path, nodule_name)).convert("1")
  mask = np.array(mask)

  subtract = Image.open(os.path.join(subtract_path, nodule_name)).convert("L")
  subtract = np.array(subtract)
  subtract[mask == False] = 0

  subtract = Image.fromarray(subtract).convert("L")
  mask = Image.new(mode="L", size=(SIZE, SIZE))
  rect = ImageDraw.Draw(mask)
  rect.rectangle(bbox, fill = 255)

  mask.save(os.path.join("./better_dataset/generated_masks", nodule_name))
  subtract.save(os.path.join("./better_dataset/subtracts", nodule_name))

In [11]:
for i in range(16):
  width = random.randint(30, 100)
  height = random.randint(30, 100)

  xmin, ymin, xmax, ymax = SIZE//2 - width//2, SIZE//2 - height//2, SIZE//2 + width//2, SIZE//2 + height//2

  mask = np.zeros(shape=(SIZE, SIZE))
  mask[ymin:ymax, xmin:xmax] = 255

  subtract = Image.new(mode="L", size=(SIZE, SIZE))
  mask = Image.fromarray(mask).convert("L")

  mask.save(os.path.join("./test/generated_masks", f"{i}.jpg"))
  subtract.save(os.path.join("./test/subtracts", f"{i}.jpg"))

In [34]:
from dataclasses import dataclass


@dataclass
class TrainingConfig:
    image_size = 256  # the generated image resolution
    train_batch_size = 8
    eval_batch_size = 16  # how many images to sample during evaluation
    num_epochs = 50
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 10
    mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "ddpm-nodules-128"  # the model name locally and on the HF Hub

    push_to_hub = False  # whether to upload the saved model to the HF Hub
    hub_private_repo = False
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 0


config = TrainingConfig()

In [35]:
test_nodule_dir = "./test/subtracts/"
test_mask_dir = "./test/generated_masks/"

test_nodules = [Image.open(os.path.join(test_nodule_dir, name)) for name in os.listdir(test_nodule_dir) if name[-1] == "g"]
test_masks = [Image.open(os.path.join(test_mask_dir, name)) for name in os.listdir(test_mask_dir) if name[-1] == "g"]

test_nodules = test_nodules[:config.eval_batch_size]
test_masks = test_masks[:config.eval_batch_size]

In [36]:
from datasets import Dataset

#config.dataset_name = "huggan/smithsonian_butterflies_subset"
#dataset = load_dataset(config.dataset_name, split="train")

nodule_dir = "./better_dataset/subtracts/"
mask_dir = "./better_dataset/generated_masks/"


data_dict = {"nodule": [], "mask": []}

for nodule_name in os.listdir(nodule_dir):
  if nodule_name not in os.listdir(test_nodule_dir):
    assert nodule_name in os.listdir(nodule_dir)
    assert nodule_name in os.listdir(mask_dir)

    data_dict["nodule"].append(Image.open(os.path.join(nodule_dir, nodule_name)))
    data_dict["mask"].append(Image.open(os.path.join(mask_dir, nodule_name)))

dataset = Dataset.from_dict(data_dict)

<Tip>

💡 You can find additional datasets from the [HugGan Community Event](https://huggingface.co/huggan) or you can use your own dataset by creating a local [`ImageFolder`](https://huggingface.co/docs/datasets/image_dataset#imagefolder). Set `config.dataset_name` to the repository id of the dataset if it is from the HugGan Community Event, or `imagefolder` if you're using your own images.

</Tip>

🤗 Datasets uses the [Image](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Image) feature to automatically decode the image data and load it as a [`PIL.Image`](https://pillow.readthedocs.io/en/stable/reference/Image.html) which we can visualize:

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 4, figsize=(16, 4))
for i, image in enumerate(dataset[:4]["nodule"]):
    axs[i].imshow(image, cmap='gray')
    axs[i].set_axis_off()
fig.show()

fig, axs = plt.subplots(1, 4, figsize=(16, 4))
for i, image in enumerate(dataset[:4]["mask"]):
    axs[i].imshow(image, cmap='gray')
    axs[i].set_axis_off()
fig.show()

In [38]:
from torchvision import transforms

image_preprocess = transforms.Compose(
    [
        transforms.Resize((config.image_size, config.image_size)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

mask_preprocess = transforms.Compose(
    [
        transforms.Resize((config.image_size, config.image_size)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
    ]
)

Use 🤗 Datasets' [set_transform](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.set_transform) method to apply the `preprocess` function on the fly during training:

In [39]:
def transform(examples):
    images = [image_preprocess(image.convert("RGB")) for image in examples["nodule"]] # doesn't matter cause we do gray scale in transform
    masks = [mask_preprocess(image.convert("RGB")) for image in examples["mask"]]
    return {"images": images, "masks": masks}


dataset.set_transform(transform)

test_nodules = torch.stack([image_preprocess(i) for i in test_nodules])
test_masks = torch.stack([mask_preprocess(i) for i in test_masks])

Feel free to visualize the images again to confirm that they've been resized. Now you're ready to wrap the dataset in a [DataLoader](https://pytorch.org/docs/stable/data#torch.utils.data.DataLoader) for training!

In [40]:
# custom loss

def weighted_mse_loss(input, target, batch_weights):
  batch_weighs = batch_weights.view(config.train_batch_size, 1, 1, 1)
  return torch.mean(batch_weighs * (input - target) ** 2)

In [41]:
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

In [ ]:
assert test_nodules.shape == test_masks.shape

image = torch.randn(test_nodules.shape)

image = image * test_masks + (1-test_masks) * test_nodules


image = image[3]

d = ((image + 1.0) * 127.5).type(torch.uint8)[0]
transforms.functional.to_pil_image(d)

## Create a UNet2DModel

Pretrained models in 🧨 Diffusers are easily created from their model class with the parameters you want. For example, to create a [UNet2DModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d#diffusers.UNet2DModel):

In [43]:
from diffusers import UNet2DModel

model = UNet2DModel(
    sample_size=config.image_size,  # the target image resolution
    in_channels=1,  # the number of input channels, 3 for RGB images
    out_channels=1,  # the number of output channels
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channels for each UNet block
    down_block_types=(
        "DownBlock2D",  # a regular ResNet downsampling block
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",  # a regular ResNet upsampling block
        "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
)

It is often a good idea to quickly check the sample image shape matches the model output shape:

In [44]:
sample_image = dataset[0]["images"].unsqueeze(0)
print("Input (image) shape:", sample_image.shape)

Input (image) shape: torch.Size([1, 1, 256, 256])


In [45]:
sample_mask = dataset[0]["masks"].unsqueeze(0)
print("Input (mask) shape:", sample_mask.shape)

Input (mask) shape: torch.Size([1, 1, 256, 256])


In [46]:
print("Output shape:", model(sample_image, timestep=0).sample.shape)

Output shape: torch.Size([1, 1, 256, 256])


Great! Next, you'll need a scheduler to add some noise to the image.

## Create a scheduler

The scheduler behaves differently depending on whether you're using the model for training or inference. During inference, the scheduler generates image from the noise. During training, the scheduler takes a model output - or a sample - from a specific point in the diffusion process and applies noise to the image according to a *noise schedule* and an *update rule*.

Let's take a look at the [DDPMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/ddpm#diffusers.DDPMScheduler) and use the `add_noise` method to add some random noise to the `sample_image` from before:

In [47]:
import torch
from PIL import Image
from diffusers import DDIMScheduler

noise_scheduler = DDIMScheduler(num_train_timesteps=1000)

In [ ]:
noise = torch.randn(sample_image.shape)

timesteps = torch.LongTensor([999])
noisy_image = sample_mask * noise_scheduler.add_noise(sample_image, noise, timesteps) + (1 - sample_mask) * sample_image
# noisy_image = noise_scheduler.add_noise(sample_image, noise, timesteps)

print(noisy_image.shape)

d = ((noisy_image + 1.0) * 127.5).type(torch.uint8)[0]
transforms.functional.to_pil_image(d)

In [49]:
import torch.nn.functional as F

# ignore error some goofy ahh device thing
noise_pred = model(noisy_image, timesteps).sample
loss = F.mse_loss(noise_pred, noise)

## Train the model

By now, you have most of the pieces to start training the model and all that's left is putting everything together.

First, you'll need an optimizer and a learning rate scheduler:

In [50]:
from diffusers.optimization import get_cosine_schedule_with_warmup

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

Then, you'll need a way to evaluate the model. For evaluation, you can use the [DDPMPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/ddpm#diffusers.DDPMPipeline) to generate a batch of sample images and save it as a grid:

In [51]:
from diffusers import DDIMPipeline
import math
import os


def make_grid(images, rows, cols):
    w, h = images[0].size
    grid = Image.new("L", size=(cols * w, rows * h))
    for i, image in enumerate(images):
        grid.paste(image, box=(i % cols * w, i // cols * h))
    return grid


def evaluate(config, epoch, pipeline):
    # Sample some images from random noise (this is the backward diffusion process).
    # The default pipeline output type is `List[PIL.Image]`
    images = pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.manual_seed(config.seed),
        masks=test_masks,
        nodules=test_nodules
    ).images

    # Make a grid out of the images
    image_grid = make_grid(images, rows=4, cols=4)

    # Save the images
    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")

Now you can wrap all these components together in a training loop with 🤗 Accelerate for easy TensorBoard logging, gradient accumulation, and mixed precision training. To upload the model to the Hub, write a function to get your repository name and information and then push it to the Hub.

<Tip>

💡 The training loop below may look intimidating and long, but it'll be worth it later when you launch your training in just one line of code! If you can't wait and want to start generating images, feel free to copy and run the code below. You can always come back and examine the training loop more closely later, like when you're waiting for your model to finish training. 🤗

</Tip>

In [52]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [53]:
from accelerate import Accelerator
from huggingface_hub import HfFolder, Repository, whoami
from tqdm.auto import tqdm
from pathlib import Path
import os


def get_full_repo_name(model_id: str, organization: str = None, token: str = None):
    if token is None:
        token = HfFolder.get_token()
    if organization is None:
        username = whoami(token)["name"]
        return f"{username}/{model_id}"
    else:
        return f"{organization}/{model_id}"


def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize accelerator and tensorboard logging
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )
    if accelerator.is_main_process:
        if config.push_to_hub:
            repo_name = get_full_repo_name(Path(config.output_dir).name)
            repo = Repository(config.output_dir, clone_from=repo_name)
        elif config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        accelerator.init_trackers("train_example")

    # Prepare everything
    # There is no specific order to remember, you just need to unpack the
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch["images"]
            masks = batch["masks"]

            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape).to(clean_images.device)

            #batch_weights = (config.image_size**2) / torch.sum(masks, dim=[1, 2, 3])

            bs = clean_images.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device
            ).long()

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_images = masks * noise_scheduler.add_noise(clean_images, noise, timesteps) + (1-masks) * clean_images

            with accelerator.accumulate(model):
                # Predict the noise residual
                noise_pred = model(noisy_images, timesteps, return_dict=False)[0] * masks
                noise *= masks
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        # After each epoch you optionally sample some demo images with evaluate() and save the model
        if accelerator.is_main_process:
            pipeline = DDIMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                evaluate(config, epoch, pipeline)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                if config.push_to_hub:
                    repo.push_to_hub(commit_message=f"Epoch {epoch}", blocking=True)
                else:
                    pipeline.save_pretrained(config.output_dir)

Phew, that was quite a bit of code! But you're finally ready to launch the training with 🤗 Accelerate's [notebook_launcher](https://huggingface.co/docs/accelerate/main/en/package_reference/launchers#accelerate.notebook_launcher) function. Pass the function the training loop, all the training arguments, and the number of processes (you can change this value to the number of GPUs available to you) to use for training:

In [ ]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)

Once training is complete, take a look at the final 🦋 images 🦋 generated by your diffusion model!

## Next steps

Unconditional image generation is one example of a task that can be trained. You can explore other tasks and training techniques by visiting the [🧨 Diffusers Training Examples](https://huggingface.co/docs/diffusers/main/en/tutorials/../training/overview) page. Here are some examples of what you can learn:

* [Textual Inversion](https://huggingface.co/docs/diffusers/main/en/tutorials/../training/text_inversion), an algorithm that teaches a model a specific visual concept and integrates it into the generated image.
* [DreamBooth](https://huggingface.co/docs/diffusers/main/en/tutorials/../training/dreambooth), a technique for generating personalized images of a subject given several input images of the subject.
* [Guide](https://huggingface.co/docs/diffusers/main/en/tutorials/../training/text2image) to finetuning a Stable Diffusion model on your own dataset.
* [Guide](https://huggingface.co/docs/diffusers/main/en/tutorials/../training/lora) to using LoRA, a memory-efficient technique for finetuning really large models faster.